In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re, threading
import os
import requests
from datetime import datetime

In [2]:
def save_full_screenshot(driver, regnumber=""):
    try:
        os.makedirs("screenshots", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"screenshots/{regnumber}_{timestamp}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")

In [3]:
def scrape(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    
    # =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
#     try:
#         # get the login button
#         login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@data-qa="navbar login button"]')))
#         if login:
#             driver.execute_script("arguments[0].scrollIntoView();", login)
#             login.click()
#         else:
#             pass

#     except Exception as e:
#         print(f"No login tab found and the error is {e}")

# # =====================================================================================
#     # complete logiin info
# # =====================================================================================

#     # get the username tab
#     try:
#         provided_u_name = "fourbrotherstrading@icloud.com"            
#         user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'.//input[@id="email"]')))   
#         user_name.send_keys(provided_u_name)
#     except Exception as e:
#         print(f"No username tab found and the error is {e}")

#     # get password tab
#     try:
#         provided_pass = "Muhssan7865#"
#         password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@id="password"]')))
#         password.send_keys(provided_pass)
#     except Exception as e:
#         print(f"No password tab found and the error is {e}")
        
    # click login
    # try:
    #     log = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@data-qa="login form button"]')))
    #     if log:
    #         log.click()
    #         # time.sleep(5)
    # except Exception as e:
    #     print("No log in")
    
    # =====================================================================================
    # handle cookies
    # =====================================================================================
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')))
        if cookie:
            cookie.click()
        else:
            pass
    except Exception as e:
        print("No cookies tab found and the error is {e}")
        
    
    # =====================================================================================
    # get total cars
    # =====================================================================================
    # try:
    #     num_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="listing-card"]')))
    #     if num_cars:
    #         # cars_list = num_cars.split(" ")
    #         cars =  int(len(num_cars)) # get only number
    #         print(f"Total {cars} of cars found")
    # except Exception as e:
    #     print(f"No number of cars found and error is {e}")
        
    
    # result
    results = []
    # car_count = 0
    # while car_count < cars: #total_cars
        
    #     if not driver.window_handles:
    #         print("Browser window closed. Stopping script.")
    #         break
        
    #     try:
    #         page_cars= WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="listing-card"]')))
            
    #         for i in range(len(page_cars)): #total_cars
    #             if car_count >= cars:
    #                 break
                
                # try:
                #     page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="listing-card"]')))
                #     driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                #     # time.sleep(1)
                #     page_cars[i].click()
                #     time.sleep(2)

    details = {}

    # car name --> done
    try:
        title = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="hero-headline__title"]/h1'))).text.strip()
        if title:
            details['Title'] = title
        else:
            details['Title'] = "na"
        save_full_screenshot(driver,details['Title'])
    except Exception as e:
        print(f"No car title ")
    
    # end date time
    try:
        end_dt = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="end-date"]'))).text.strip()
        if end_dt:
            details['Auc_end'] = end_dt
        else:
            details['Auc_end'] = 'na'
    except Exception as e:
        print("No auc end")
    
    # current bid
    try:
        curr_bid =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@data-qa="listing highest bid"]'))).text.strip() 
        if curr_bid:
            details['Current_bid'] = curr_bid
        else:
            details['Current_bid'] = 'na'
    except Exception as e:
        print("No current bid")
    
    # no bids
    try:
        num_bids = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@data-qa="listing bid count"]'))).text.strip() 
        if num_bids:
            details['No:bids'] = num_bids
        else:
            details['No:bids'] = 'na'
    except Exception as e:
        print("No num bids")
    
    # key facts --> done
    try:
        facts_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-info-details__key-facts"]')))
        driver.execute_script("arguments[0].scrollIntoView();", facts_card)
        if facts_card:
            facts_lbl = WebDriverWait(facts_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'h3'))).text.strip()
            if facts_lbl:
                # print("Label got")
                facts_ul = WebDriverWait(facts_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'ul')))
                if facts_ul:
                    # print("Ul found")
                    facts_vals =  WebDriverWait(facts_ul, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'li')))
                    if facts_vals:
                        # print("Facts found li")
                        fact_values = [fact_val.text.strip() for fact_val in facts_vals]
                        details[facts_lbl] = fact_values
                    else:
                        print("No facts data")
    except Exception as e:
        print(f"No facts data")
    
    # other information --> done
    try:
        det_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-info-details__stats"]'))) 
        if det_card:
            # print("det found")
            det_ul = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'ul')))
            if det_ul:
                # print("ul found")
                det_vals =  WebDriverWait(det_ul, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'li')))
                if det_vals:
                    lbl_lst = []
                    for li in det_vals:
                        lbls = li.get_attribute("data-qa").split(" ")[3]
                        lbl_lst.append(lbls)
                        
                        spans = li.find_elements(By.XPATH, './/span[@class="icon-text is-capitalized"]')
                        if spans:
                            for span in spans:
                                vals = span.find_element(By.XPATH, './/span[@class="text"]').text.strip()
                                details[lbls] = vals
                    # results.append(dict(zip(lbl_lst, vals)))
    except Exception as e:
        print(f"No car dets")
        
    # location --> done
    try:
        loc_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-info-details__location"]'))) 
        if loc_card:
            loc_lbl = loc_card.find_element(By.TAG_NAME, 'h6').text.strip()
            loc_val = loc_card.find_element(By.XPATH, './/span[@class="text"]').text.strip()
            if loc_lbl and loc_val:
                details[loc_lbl] = loc_val
            else:
                print("No loc infor")
    except Exception as e:
        print("No loc infor")
        
    # bg dets --> done
    try:
        bg_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content background"])[1]'))) 
        if bg_card:
            bg_dets = WebDriverWait(bg_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
            if bg_dets:
                details['Background'] = bg_dets
            else:
                details['Background'] = 'na'
    except Exception as e:
        print("No bg card found")
        
    # overview dets --> done
    try:
        over_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content overview"])[1]'))) 
        driver.execute_script("arguments[0].scrollIntoView();", over_card)
        if over_card:
            over_dets = WebDriverWait(over_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
            if over_dets:
                details['Overview'] = over_dets
            else:
                details['Overview'] = 'na'
    except Exception as e:
        print("No overview card found")
        
    # est val and seller --> done
    try:
        est_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-about__details"]'))) 
        if est_card:
            
            # set val --> done
            try:
                est = WebDriverWait(est_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-about__estimated"]')))
                if est:
                    est_lbl = est.find_element(By.TAG_NAME, 'p').text.strip()
                    est_val = est.find_element(By.XPATH, './/span[@class="text"]').text.strip()
                    if est_lbl and est_val:
                        details[est_lbl] = est_val
            except Exception as e:
                print("No estimated values")
                
            # seller --> done
            try:
                sell = WebDriverWait(est_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-about__seller"]')))
                if sell:
                    sell_lbl = sell.find_element(By.TAG_NAME, 'p').text.strip()
                    sell_val = sell.find_element(By.XPATH, './/span[@class="text"]').text.strip()
                    if sell_lbl and sell_val:
                        details[sell_lbl] = sell_val
            except Exception as e:
                print("No seller values")
    except Exception as e:
        print("No seller dets")
        
    # Exterior dets and images --> done
    try:
        ext_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content exterior"])[1]'))) 
        driver.execute_script("arguments[0].scrollIntoView();", ext_card)
        if ext_card:
            ext_dets = WebDriverWait(ext_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
            if ext_dets:
                details['exterior'] = ext_dets
            else:
                details['exterior'] = 'na'
            
            try:
                # imgs
                to_clk = WebDriverWait(ext_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="vehicle-gallery__overlay"]'))) 
                if to_clk:
                    driver.execute_script("arguments[0].scrollIntoView();", to_clk)
                    to_clk.click()
                    time.sleep(2)
                    
                    img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@id="productGallery"])')))
                    if img_card:
                        imgs = WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class=" twic-done"]')))
                        if imgs:
                            imgs_lst = [img.get_attribute("src") for img in imgs]
                            details['Exterior_Images'] = ", ".join(imgs_lst)
                        else:
                            print("No images ")
            except Exception as e:
                print("No exterior images found")   
            driver.back()     
    except Exception as e:
        print("No exterior card found")
    

    try:
        intr_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content interior"])[1]'))) 
        driver.execute_script("arguments[0].scrollIntoView();", intr_card)
        if intr_card:
            intr_dets = WebDriverWait(intr_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
            if intr_dets:
                details['interior'] = intr_dets
            else:
                details['interior'] = 'na'
            
            try:
                intr_card = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content interior"])[1]'))
                )
                driver.execute_script("arguments[0].scrollIntoView();", intr_card)
                time.sleep(1)
                
                # Try clicking gallery if available
                try:
                    gallery_link = intr_card.find_element(By.XPATH, './/a[contains(@class,"vehicle-gallery__overlay")]')
                    gallery_link.click()
                    time.sleep(2)
                except:
                    print("No overlay to click, checking directly for images.")

                # Get all image URLs from the gallery
                imgs = driver.find_elements(By.XPATH, '//img[contains(@src, "bonhamscarsonline.twic.pics")]')
                if imgs:
                    imgs_lst = [img.get_attribute("src") for img in imgs]
                    details["interior_Images"] = ", ".join(imgs_lst)
                    print(f"✅ Found {len(imgs_lst)} interior images")
                else:
                    print("No interior images found")

            except Exception as e:
                print(f"No interior card found: {e}")

            driver.back()   
    except Exception as e:
        print("No interior card found")
            
    # Mechanical dets and images --> done
    try:
        mech_card = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content mechanical"])[1]'))
        )
        driver.execute_script("arguments[0].scrollIntoView();", mech_card)

        # Mechanical details text
        try:
            mech_dets = WebDriverWait(mech_card, 5).until(
                EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))
            ).text.strip()
            details["mechanical"] = mech_dets if mech_dets else "na"
        except:
            details["mechanical"] = "na"

        # Mechanical images
        try:
            to_clk = WebDriverWait(mech_card, 5).until(
                EC.presence_of_element_located((By.XPATH, './/a[@class="vehicle-gallery__overlay"]'))
            )
            driver.execute_script("arguments[0].scrollIntoView();", to_clk)
            to_clk.click()

            img_card = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '(.//div[@id="productGallery"])'))
            )
            imgs = WebDriverWait(img_card, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, './/img[@class=" twic-done"]'))
            )
            imgs_lst = [img.get_attribute("src") for img in imgs]
            details["mechanical_Images"] = ", ".join(imgs_lst) if imgs_lst else "na"

            driver.back()

        except:
            details["mechanical_Images"] = "na"

    except:
        details["mechanical"] = "na"
        details["mechanical_Images"] = "na"

            
    # History dets and images --> done
    try:
        hist_card = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content history"])[1]'))
        )
        driver.execute_script("arguments[0].scrollIntoView();", hist_card)

        # History text
        try:
            hist_dets = WebDriverWait(hist_card, 5).until(
                EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))
            ).text.strip()
            details["History"] = hist_dets if hist_dets else "na"
        except:
            details["History"] = "na"

        # History images
        try:
            to_clk = WebDriverWait(hist_card, 5).until(
                EC.presence_of_element_located((By.XPATH, '(.//a[@class="mt-2"])[1]'))
            )
            to_clk.click()

            img_card = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '(.//div[@id="productGallery"])'))
            )
            imgs = WebDriverWait(img_card, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, './/img[@class=" twic-done"]'))
            )
            imgs_lst = [img.get_attribute("src") for img in imgs]
            details["History_Images"] = ", ".join(imgs_lst) if imgs_lst else "na"

            driver.back()

        except:
            details["History_Images"] = "na"

    except:
        details["History"] = "na"
        details["History_Images"] = "na"

        
    # Summary dets --> done
    try:
        summary_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content summary"])[1]'))) 
        driver.execute_script("arguments[0].scrollIntoView();", summary_card)
        if summary_card:
            summary_dets = WebDriverWait(summary_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
            if summary_dets:
                details['Summary'] = summary_dets
            else:
                details['Summary'] = 'na'
    except Exception as e:
        print("No summary found")
    
    # bids
    try:
        # Wait for all bid comment cards
        comments = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, '//article[@class="comments__activity level"]'))
        )

        bids_data = []

        for comment in comments:
            try:
                username = comment.find_element(By.XPATH, './/span[contains(@class,"bid-comment__username")]').text.strip()
            except:
                username = "N/A"

            try:
                amount = comment.find_element(By.XPATH, './/span[contains(@class,"bid-comment__amount")]').text.strip()
            except:
                amount = "N/A"

            try:
                time_text = comment.find_element(By.XPATH, './/span[contains(@class,"bid-comment__time")]').text.strip()
            except:
                time_text = "N/A"

            # Combine one record
            bids_data.append(f"{username} - {amount} - {time_text}")

        if bids_data:
            # Convert list to comma separated string
            details["Bids_History"] = ", ".join(bids_data)
        else:
            details["Bids_History"] = "na"

    except Exception as e:
        print("❌ No bid history found:", e)
        details["Bids_History"] = "na"


    
    # car_count+=1
    results.append(details)
                    # driver.back()
                    
        #         except Exception as e:
        #             print("No more cars to click")
        #             # break
        # except Exception as e:
        #     print("No more cars")
        #     # break
        
    # time.sleep(5)
    df = pd.DataFrame.from_dict(results)
    df.to_csv("Bonhams.csv", index=False)
    driver.quit()

path = "https://carsonline.bonhams.com/en/listings/audi/wr-quattro/1f11b7ad-ddab-4a5c-9af9-8183c8ec1a9d"
scrape(path)

📸 Screenshot saved: screenshots/1982 Audi WR Quattro_20251125_141627.png
✅ Found 289 interior images


In [4]:
df = pd.read_csv("Bonhams.csv")
df.columns

Index(['Title', 'Auc_end', 'Current_bid', 'No:bids', 'Key Facts', 'chassis',
       'mileage', 'engine', 'gearbox', 'color', 'interior', 'steering',
       'fuel_type', 'Vehicle location', 'Background', 'Overview',
       'Estimated value', 'Seller', 'exterior', 'Exterior_Images',
       'interior_Images', 'mechanical', 'mechanical_Images', 'History',
       'History_Images', 'Summary', 'Bids_History'],
      dtype='object')

### Image downloader

### Exterior images

In [5]:
from urllib.parse import urlparse, urljoin,urlunparse

from PIL import Image, ImageDraw, ImageFont
from PIL import Image, ImageDraw, ImageFont

def add_watermark_to_image(image_path, text="Sourced from Bonhams"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # 🔹 Slightly smaller dynamic font size (half previous)
        font_size = max(20, image.width // 50)  # reduced from //25 → //50
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = int(font_size * 0.6)
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_padding = int(font_size * 0.4)
        draw.rectangle(
            [x - box_padding, y - box_padding, x + text_width + box_padding, y + text_height + box_padding],
            fill=(0, 0, 0, 180)
        )

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 240))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✅ Watermark added to {image_path}")

    except Exception as e:
        print(f"❌ Failed to watermark {image_path}: {e}")

        
df = pd.read_csv("Bonhams.csv")
ext_img = df[["Title", "Exterior_Images"]]
int_img = df[["Title", "interior_Images"]]
mech_img = df[["Title", "mechanical_Images"]]
# reg_img = df[["Title", "History_Images"]]
# hist_img = df[["Title", "History_Images"]]
def clean_image_url(url):
    """
    Removes resizing or twic query parameters from the image URL.
    Example:
    https://example.com/image.jpg?twic=v1/resize=650 -> https://example.com/image.jpg
    """
    return url.split("?")[0].strip()

def download_images(data, column_name, main_folder):
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Title"]
        image_urls = row[column_name]

        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)

        for idx, url in enumerate(image_urls):
            url = clean_image_url(url.strip())
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)

            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue

            try:
                response = requests.get(url, stream=True, timeout=10)
                response.raise_for_status()

                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1].lower()
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"

                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                print(f"✅ Downloaded: {full_file_name}")
                add_watermark_to_image(full_file_name)  # 💧 Add watermark after saving

            except Exception as e:
                print(f"❌ Failed to download {url} for {reg_no}: {e}")

# ✅ Specific Wrappers for each image type
def download_ext(data): 
    download_images(data, "Exterior_Images", "Exterior_Images")

def download_int(data): 
    download_images(data, "interior_Images", "interior_Images")

def download_mech(data): 
    download_images(data, "mechanical_Images", "mechanical_Images")

# hist images 
# def download_hist(data, main_folder="History_Images"): 
#     os.makedirs(main_folder, exist_ok=True)
    
#     for index, row in data.iterrows():
#         reg_no = row["Title"] 
#         image_urls = row["History_Images"]

#         # Check if 'Images' column is empty or NaN
#         if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
#             print(f"Skipping {reg_no} (No image URLs)")
#         else:
#             image_urls = image_urls.split(", ")  # Split URLs by comma
            
#             reg_folder = os.path.join(main_folder, reg_no)
#             os.makedirs(reg_folder, exist_ok=True)
            
#             for idx, url in enumerate(image_urls):
#                 url = url.strip()
#                 if not url.startswith(("http://", "https://")):
#                     url = urljoin("https://", url) 
                
#                 parsed_url = urlparse(url)
#                 if not parsed_url.scheme or not parsed_url.netloc:
#                     print(f"Invalid URL skipped: {url}") 
#                     continue
                
#                 try:
#                     response = requests.get(url, stream=True) 
#                     response.raise_for_status()
                    
#                     file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
#                     file_extension = file_name.split(".")[-1]
                    
#                     if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
#                         file_name = f"image_{idx + 1}.jpg"
                    
#                     full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                    
#                     with open(full_file_name, 'wb') as f:
#                         for chunk in response.iter_content(1024):
#                             f.write(chunk)
                    
#                     print(f"Downloaded: {full_file_name}")
#                 except Exception as e:
#                     print(f"Failed to download {url} for {reg_no}: {e}")

def start_funcs():
    thread1 = threading.Thread(target=download_ext, args=(ext_img,))
    thread2 = threading.Thread(target=download_int, args=(int_img,))
    thread3 = threading.Thread(target=download_mech, args=(mech_img,))
    # thread4 = threading.Thread(target=download_hist, args=(hist_img,))
    
    thread1.start()
    thread2.start()
    thread3.start()
    # thread4.start()
    
    thread1.join()
    thread2.join()
    thread3.join() 
    # thread4.join()

if __name__ == "__main__":
    start_funcs()   

Invalid URL skipped: https:///na
✅ Downloaded: Exterior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_1_6892ff12-0af0-4c0e-82c1-553697a94b88.jpg
✅ Downloaded: interior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_1_941c0646-b9a4-4a1a-9e0f-a04cec31892a.jpg
✅ Watermark added to Exterior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_1_6892ff12-0af0-4c0e-82c1-553697a94b88.jpg
✅ Watermark added to interior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_1_941c0646-b9a4-4a1a-9e0f-a04cec31892a.jpg
✅ Downloaded: interior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_2_6892ff12-0af0-4c0e-82c1-553697a94b88.jpg
✅ Watermark added to interior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_2_6892ff12-0af0-4c0e-82c1-553697a94b88.jpg
✅ Downloaded: Exterior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_2_1ae0212e-e935-4670-ac77-6d54ad22bacc.jpg
✅ Downloaded: interior_Images\1982 Audi WR Quattro\1982 Audi WR Quattro_3_1ae0212e-e935-4670-ac77-6d54ad22bacc.jpg
✅ Watermark added to Exter